In [ ]:
import csv
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
import seaborn as sns
import os.path
import time
import functions

In [ ]:
filepath ='data/'
csvfilenames = ['confinement_training.csv', 'member_information.csv', 'labresults_training.csv', 'medical_training.csv', 'rx_training.csv']
txtfilenames = ['lu_diagnosis.txt', 'lu_procedure-fake.txt', 'lu_ndc.txt' ]

"""with open(filepath+csvfilenames[0]) as datafile:
    filecontents = csv.reader(datafile, delimiter = ",")
    data = list(filecontents)

confinement = pd.DataFrame(data)
confinement.columns = confinement.iloc[0]
confinement = confinement[1:] #sets first row as header"""

with open(filepath+csvfilenames[1]) as datafile:
    filecontents = csv.reader(datafile, delimiter = ",")
    data = list(filecontents)

members = pd.DataFrame(data)
members.columns = members.iloc[0]
members = members[1:]
members = members.set_index(['PATID']) #makes PATID the row index instead of integers, not sure it's worth it.

with open(filepath+csvfilenames[2]) as datafile:
    filecontents = csv.reader(datafile, delimiter = ",")
    data = list(filecontents)

labdata = pd.DataFrame(data)
labdata.columns = labdata.iloc[0]
labdata = labdata[1:]

with open(filepath+csvfilenames[3]) as datafile:
    filecontents = csv.reader(datafile, delimiter = ",")
    data = list(filecontents)

allmedical = pd.DataFrame(data)
allmedical.columns = allmedical.iloc[0]
allmedical = allmedical[1:]

with open(filepath+csvfilenames[4]) as datafile:
    filecontents = csv.reader(datafile, delimiter = ",")
    data = list(filecontents)

rxdata = pd.DataFrame(data)
rxdata.columns = rxdata.iloc[0]
rxdata = rxdata[1:]

In [ ]:
allmedical['STD_COST']=allmedical['STD_COST'].apply(pd.to_numeric) #change entries to numbers
rxdata.loc[rxdata['STD_COST']=='None', 'STD_COST']=0 #change all nones to zeros
rxdata['STD_COST']=rxdata['STD_COST'].apply(pd.to_numeric) #change all entries to numbers
members['year_of_birth']=members['year_of_birth'].apply(pd.to_numeric)
rxdata['DAYS_FROM_DIAG'] = rxdata['DAYS_FROM_DIAG'].apply(pd.to_numeric)
medical_wanted['DAYS_FROM_DIAG'] = medical_wanted['DAYS_FROM_DIAG'].apply(pd.to_numeric)

In [ ]:
allmedical['wanted']=True #this is unecessary?

medical_wanted = allmedical[('630' >=  allmedical.DIAG1) | (allmedical.DIAG1 >= '67999')]
medical_wanted = medical_wanted[('630' >=  medical_wanted.DIAG2) | (medical_wanted.DIAG2 >= '67999')]

medical_wanted = medical_wanted[('800' >=  medical_wanted.DIAG1) | (medical_wanted.DIAG1 >= '95999')]
medical_wanted = medical_wanted[('800' >=  medical_wanted.DIAG2) | (medical_wanted.DIAG2 >= '95999')]

medical_wanted = medical_wanted[medical_wanted['DIAG1'].str.startswith('E') == False]
medical_wanted = medical_wanted[medical_wanted['DIAG2'].str.startswith('E') == False]

medical_wanted = medical_wanted[(medical_wanted['DIAG1'].str.startswith('V3') == False) & (medical_wanted['DIAG1'].str.startswith('V22') == False) & (medical_wanted['DIAG1'].str.startswith('V23') == False) & (medical_wanted['DIAG1'].str.startswith('V24') == False) & (medical_wanted['DIAG1'].str.startswith('V28') == False) & (medical_wanted['DIAG1'].str.startswith('V91') == False)]
medical_wanted = medical_wanted[(medical_wanted['DIAG2'].str.startswith('V3') == False) & (medical_wanted['DIAG2'].str.startswith('V22') == False) & (medical_wanted['DIAG2'].str.startswith('V23') == False) & (medical_wanted['DIAG2'].str.startswith('V24') == False) & (medical_wanted['DIAG2'].str.startswith('V28') == False) & (medical_wanted['DIAG2'].str.startswith('V91') == False)]


In [ ]:
wantedmedical = allmedical[allmedical.wanted]

In [ ]:
members['totalcost'] = 0.0 #add rxdata to member's total cost
members['rxcost'] = 0.0
members['medicalcost'] = 0.0

dummy = rxdata.groupby('PATID')['STD_COST'].sum()
#print dummy

for k in range(len(dummy)):
    members.at[dummy.index[k],'totalcost'] += dummy.iat[k]
    members.at[dummy.index[k], 'rxcost'] = dummy.iat[k]
    #print type(members.at[dummy.index[k],'totalcost'])
    
dummy = medical_wanted.groupby('PATID')['STD_COST'].sum()
#print dummy.head()

for k in range(len(dummy)):
    members.at[dummy.index[k], 'totalcost'] += dummy.iat[k]
    members.at[dummy.index[k], 'medicalcost'] = dummy.iat[k]
    
#print members.head()


In [ ]:
#all potentially completely useless
dummy = rxdata.groupby('PATID')['DAYS_FROM_DIAG'].min()
for k in range(len(dummy)):
    members.at[dummy.index[k], 'minrxday'] = dummy.iat[k]
dummy = rxdata.groupby('PATID')['DAYS_FROM_DIAG'].max()
for k in range(len(dummy)):
    members.at[dummy.index[k], 'maxrxday'] = dummy.iat[k]
dummy = medical_wanted.groupby('PATID')['DAYS_FROM_DIAG'].min()
for k in range(len(dummy)):
    members.at[dummy.index[k], 'minmedicalday'] = dummy.iat[k]
dummy = medical_wanted.groupby('PATID')['DAYS_FROM_DIAG'].max()
for k in range(len(dummy)):
    members.at[dummy.index[k], 'maxmedicalday'] = dummy.iat[k]

In [ ]:
#The next several blocks add counters to the member dataframe for the number of visits to different types of facilities

In [ ]:
#emergency=['23','24', '21', '41', '20', '42']
medical_wanted_emergency = medical_wanted[(medical_wanted['POS'] =='23') | 
                                          (medical_wanted['POS'] =='24') | 
                                          (medical_wanted['POS'] =='21') |
                                          (medical_wanted['POS'] =='41') |
                                          (medical_wanted['POS'] =='20') |
                                          (medical_wanted['POS'] =='42') ]
members['emervisits']=0
patids = list(members.index)
tick = time.clock()
patidgroups = medical_wanted_emergency.groupby('PATID')
for k in patids:
    try:
        members.loc[k, 'emervisits'] = len(patidgroups.get_group(k).groupby('DAYS_FROM_DIAG'))
    except KeyError:
        pass
tock = time.clock()
print tock-tick

In [ ]:
#psychiatric=['53', '55', '51', '52', '57'] # 51 is inpatient faciliity, 55 is res. sub. abuse
medical_wanted_psych = medical_wanted[(medical_wanted['POS'] =='53') | 
                                          #(medical_wanted['POS'] =='55') |
                                          (medical_wanted['POS'] =='52') |
                                          (medical_wanted['POS'] =='57')]
members['psychvisits']=0
patids = list(members.index)
tick = time.clock()
patidgroups = medical_wanted_psych.groupby('PATID')
for k in patids:
    try:
        members.loc[k, 'psychvisits'] = len(patidgroups.get_group(k).groupby('DAYS_FROM_DIAG'))
    except KeyError:
        pass
tock = time.clock()
print tock-tick

In [ ]:
#regular=['11', '01', '81','22', '60', '49', '71', '62', '17', '50', '61', '03']
medical_wanted_office = medical_wanted[(medical_wanted['POS'] =='11') | 
                                          (medical_wanted['POS'] =='01') | 
                                          (medical_wanted['POS'] =='81') |
                                          (medical_wanted['POS'] =='22') |
                                          (medical_wanted['POS'] =='60') |
                                      (medical_wanted['POS'] =='49') | 
                                          (medical_wanted['POS'] =='71') |
                                          (medical_wanted['POS'] =='62') |
                                          (medical_wanted['POS'] =='17') |
                                      (medical_wanted['POS'] =='50') | 
                                          (medical_wanted['POS'] =='61') |
                                          (medical_wanted['POS'] =='03') ]
members['officevisits']=0
patids = list(members.index)
tick = time.clock()
patidgroups = medical_wanted_office.groupby('PATID')
for k in patids:
    try:
        members.loc[k, 'officevisits'] = len(patidgroups.get_group(k).groupby('DAYS_FROM_DIAG'))
    except KeyError:
        pass
tock = time.clock()
print tock-tick

In [ ]:
#elderly=['13', '31', '32', '33']
medical_wanted_elderly = medical_wanted[(medical_wanted['POS'] =='13') | 
                                          (medical_wanted['POS'] =='31') | 
                                          (medical_wanted['POS'] =='32') |
                                          (medical_wanted['POS'] =='33') ]
members['elderlyvisits']=0
patids = list(members.index)
tick = time.clock()
patidgroups = medical_wanted_elderly.groupby('PATID')
for k in patids:
    try:
        members.loc[k, 'elderlyvisits'] = len(patidgroups.get_group(k).groupby('DAYS_FROM_DIAG'))
    except KeyError:
        pass
tock = time.clock()
print tock-tick

In [ ]:
#home=['12', '14', '15']
medical_wanted_home = medical_wanted[(medical_wanted['POS'] =='12') | 
                                          (medical_wanted['POS'] =='14') | 
                                          (medical_wanted['POS'] =='15')]
members['homevisits']=0
patids = list(members.index)
tick = time.clock()
patidgroups = medical_wanted_home.groupby('PATID')
for k in patids:
    try:
        members.loc[k, 'homevisits'] = len(patidgroups.get_group(k).groupby('DAYS_FROM_DIAG'))
    except KeyError:
        pass
tock = time.clock()
print tock-tick

In [ ]:
#dialysis=['65']
medical_wanted_dialysis = medical_wanted[(medical_wanted['POS'] =='65')]
members['dialysisvisits']=0
patids = list(members.index)
tick = time.clock()
patidgroups = medical_wanted_dialysis.groupby('PATID')
for k in patids:
    try:
        members.loc[k, 'dialysisvisits'] = len(patidgroups.get_group(k).groupby('DAYS_FROM_DIAG'))
    except KeyError:
        pass
tock = time.clock()
print tock-tick

In [ ]:
#hospice=['34']
medical_wanted_hospice = medical_wanted[(medical_wanted['POS'] =='34')]
members['hospicevisits']=0
patids = list(members.index)
tick = time.clock()
patidgroups = medical_wanted_hospice.groupby('PATID')
for k in patids:
    try:
        members.loc[k, 'hospicevisits'] = len(patidgroups.get_group(k).groupby('DAYS_FROM_DIAG'))
    except KeyError:
        pass
tock = time.clock()
print tock-tick

In [ ]:
#unknown=['99', 'None']
medical_wanted_unknown = medical_wanted[(medical_wanted['POS'] =='99')|
                                        (medical_wanted['POS'] == 'None')]
members['unknownvisits']=0
patids = list(members.index)
tick = time.clock()
patidgroups = medical_wanted_unknown.groupby('PATID')
for k in patids:
    try:
        members.loc[k, 'unknownvisits'] = len(patidgroups.get_group(k).groupby('DAYS_FROM_DIAG'))
    except KeyError:
        pass
tock = time.clock()
print tock-tick

In [ ]:
#sample scatter plot because why not
colors = {'F': 'red', 'M': 'blue', 'U': 'green'}
members.plot.scatter(x ='emervisits', y = 'totalcost', c = members['gender'].apply(lambda x: colors[x]))
#plt.ylim(0, 2000000) #not the actual upper limit
plt.show()

In [ ]:
location_series = medical_wanted.groupby('POS')['STD_COST'].mean()

location_series.plot.bar()
plt.ylim([0,2000])
plt.show()

In [ ]:
# Add new column for region (should find a way to avoid this with groupby, but can't currently)
members['region'] = members.apply(lambda row: functions.US_region(row), axis=1)

region_series = members.groupby('region')['totalcost'].mean()

region_series.plot.bar()
plt.show()

In [ ]:
reload(functions)